In [16]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import binom
from scipy.stats import poisson
from scipy.stats import wasserstein_distance
from tqdm import tqdm
import time
import pandas as pd 

In [17]:
#Paris dataset
gowalla_paris_all = pd.read_csv (r'/Users/sayanbiswas/Desktop/Work/Projects/BAandIBU/Latest/Gowalla_Paris_grid.csv') #Importing Gowalla Paris (grid aggregated) dataset
gowalla_paris_values=gowalla_paris_all.values #Converting gowalla dataset to an array
N_paris=len(gowalla_paris_values) #Number of data points in Paris dataset

paris_loc=np.rint(gowalla_paris_values) #Rounding off latitudes and longitudes
paris_latitudes=np.rint(gowalla_paris_values[:,0]) #Latitudes rounded to nearest integer78
paris_longitudes=np.rint(gowalla_paris_values[:,1]) #Longitudes rounded to nearest integer

#Range of Paris dataset
paris_lat_max=np.max(paris_latitudes) #Maximum latitude
paris_lat_min=np.min(paris_latitudes) #Minimum latitude
paris_long_max=np.max(paris_longitudes) #Maximum longitude
paris_long_min=np.min(paris_longitudes) #Minimum longitude
paris_lat_range=np.array([paris_lat_min,paris_lat_max+1]) #Range of latitude
paris_long_range=np.array([paris_long_min,paris_long_max+1]) #Range of longitude


In [18]:
#San Francisco dataset
gowalla_sf_all = pd.read_csv (r'/Users/sayanbiswas/Desktop/Work/Projects/BAandIBU/Latest/Gowalla_SF_grid.csv') #Importing Gowalla SF (grid aggregated) dataset
gowalla_sf_values=gowalla_sf_all.values #Converting gowalla dataset to an array
N_sf=len(gowalla_sf_values) #Number of data points in SF dataset

sf_loc=np.rint(gowalla_sf_values) #Rounding off latitudes and longitudes
sf_latitudes=np.rint(gowalla_sf_values[:,0]) #Latitudes rounded to nearest integer78
sf_longitudes=np.rint(gowalla_sf_values[:,1]) #Longitudes rounded to nearest integer

#Range of SF dataset
sf_lat_max=np.max(sf_latitudes) #Maximum latitude
sf_lat_min=np.min(sf_latitudes) #Minimum latitude
sf_long_max=np.max(sf_longitudes) #Maximum longitude
sf_long_min=np.min(sf_longitudes) #Minimum longitude
sf_lat_range=np.array([sf_lat_min,sf_lat_max+1]) #Range of latitude
sf_long_range=np.array([sf_long_min,sf_long_max+1]) #Range of longitude


In [19]:
#Setup for Paris experiment

X_paris_lat=np.array(range(int(paris_lat_min),int(paris_lat_max+1)))    #Space of latitudes of original data in Paris dataset
X_paris_long=np.array(range(int(paris_long_min),int(paris_long_max+1))) #Space of longitudes of original data in Paris dataset
Y_paris_lat=np.array(range(int(paris_lat_min),int(paris_lat_max+1)))    #Space of latitudes of sanitized data in Paris dataset
Y_paris_long=np.array(range(int(paris_long_min),int(paris_long_max+1))) #Space of longitudes of sanitized data in Paris dataset
N_X_paris_lat=len(X_paris_lat) #Total length of the space of source latitudes of Paris dataset
N_X_paris_long=len(X_paris_long) #Total length of the space of source longitudes of Paris dataset
from itertools import product
X_paris=list(product(X_paris_lat, X_paris_long)) #all possible inpput pairs of (lat,long) of Paris datset
Y_paris=list(product(Y_paris_lat, Y_paris_long)) #all possible output pairs of (lat,long) of Paris dataset

N_X_paris=len(X_paris) #Total length of the space of Paris data

#Entire domain for Paris locations
X_paris_possible=len(X_paris_lat)*len(X_paris_long) #Total possible input locations of Paris dataset
Y_paris_possible=len(Y_paris_lat)*len(Y_paris_long) #Total possible output locations of Paris dataset


In [20]:
#Setup for SF experiment

X_sf_lat=np.array(range(int(sf_lat_min),int(sf_lat_max+1)))    #Space of latitudes of original data in SF dataset
X_sf_long=np.array(range(int(sf_long_min),int(sf_long_max+1))) #Space of longitudes of original data in SF dataset
Y_sf_lat=np.array(range(int(sf_lat_min),int(sf_lat_max+1)))    #Space of latitudes of sanitized data in SF dataset
Y_sf_long=np.array(range(int(sf_long_min),int(sf_long_max+1))) #Space of longitudes of sanitized data in SF dataset
N_X_sf_lat=len(X_sf_lat) #Total length of the space of source latitudes of SF dataset
N_X_sf_long=len(X_sf_long) #Total length of the space of source longitudes of SF dataset
from itertools import product
X_sf=list(product(X_sf_lat, X_sf_long)) #all possible inpput pairs of (lat,long) of SF datset
Y_sf=list(product(Y_sf_lat, Y_sf_long)) #all possible output pairs of (lat,long) of SF dataset


N_X_sf=len(X_sf) #Total length of the space of SF data

#Entire domain for SF locations
X_sf_possible=len(X_sf_lat)*len(X_sf_long) #Total possible input locations of SF dataset
Y_sf_possible=len(Y_sf_lat)*len(Y_sf_long) #Total possible output locations of SF dataset


In [21]:
###Expanding the co-domain (to implement Laplace)
#Paris
X_paris_lat_wide=np.array(range(int(paris_lat_min)-5,int(paris_lat_max+1)+5))    #Wider space of latitudes of original data in Paris dataset
X_paris_long_wide=np.array(range(int(paris_long_min)-5,int(paris_long_max+1)+5)) #Wider space of longitudes of original data in Paris dataset
X_paris_wide=list(product(X_paris_lat_wide, X_paris_long_wide)) #Wider space of all possible inpput pairs of (lat,long) of SF datset

#SF
X_sf_lat_wide=np.array(range(int(sf_lat_min)-5,int(sf_lat_max+1)+5))    #Wider space of latitudes of original data in SF dataset
X_sf_long_wide=np.array(range(int(sf_long_min)-5,int(sf_long_max+1)+5)) #Wider space of longitudes of original data in SF dataset
X_sf_wide=list(product(X_sf_lat_wide, X_sf_long_wide)) #Wider space of all possible inpput pairs of (lat,long) of SF datset


In [22]:
#Assigning each (latitude,longitude) pair it's positional index
#Converting between 1D pos to 2D val 
#FOR PARIS
def PosToValParis(t,XLat=X_paris_lat,XLong=X_paris_long):
    n=len(XLong)
    #First giving positions of corresponding latitude, longitude
    pos_lat=int((t-(t%n))/n)
    pos_long=int(t%n)
    #Note: (pos_lat,pos_long) is NOT (lat,long)
    lat=int(XLat[pos_lat])
    long=int(XLong[pos_long])
    return lat, long

def ValToPosParis(a,b,XLong=X_paris_long):
    n=len(XLong)
    return (n*a+b)


#FOR SF
def PosToValSF(t,XLat=X_sf_lat,XLong=X_sf_long):
    n=len(XLong)
    #First giving positions of corresponding latitude, longitude
    pos_lat=int((t-(t%n))/n)
    pos_long=int(t%n)
    #Note: (pos_lat,pos_long) is NOT (lat,long)
    lat=int(XLat[pos_lat])
    long=int(XLong[pos_long])
    return lat, long

def ValToPosSF(a,b,XLong=X_sf_long):
    n=len(XLong)
    return (n*a+b)


In [23]:
#Blahut-Arimoto Algorithm for Paris
def BlahutArimotoParis(C0,p0,B,R_BA,Num_X=N_X_paris,Num_Y=N_X_paris):
    ##Inputs:
        #C0=Starting Channel 
        #p0=PDF of source data
        #B=rate-distortion parameter (high values of B implies low avg distortion=QoS and high MI=privacy)
        #R_BA=Number of iterations 
        #Num_X=length of source alphabet
        #Num_Y=length of image alphabet
    ##Output:
        #Returns a channel that computes the minimum MI for a given average distortion

    start_time = time.time()
    #Intermediate storage
    Channels=[] #Storing all the channels till convergence 
    Channels.append(C0) #The first element is the initial matrix

    exp_mat=np.zeros((Num_X,Num_Y)) #Exponential matrix pre-computed 
    for x in range(Num_X):
        x_val=np.array([PosToValParis(x)[0],PosToValParis(x)[1]]) #(lat,long) value at index x
        for y in range(Num_Y):
            y_val=np.array([PosToValParis(y)[0],PosToValParis(y)[1]]) #(lat,long) value at index y
            exp_mat[x,y]=np.exp(-B*(np.linalg.norm(x_val-y_val)))

    for i in range(R_BA): #starting the iteration of BA algorithm
        #print("BA iteration:", i)
        c=np.matmul(p0,Channels[i]) #Computing the output probabilities
        C_next=np.zeros((Num_X,Num_Y)) #Creating a dummy matrix at each step
        for x in range(Num_X): #Rows of the channel
            #print("BA Channel row: ",x)
            x_val=np.array([PosToValParis(x)[0],PosToValParis(x)[1]]) #(lat,long) value at index x
            for y in range(Num_Y): #Columns of the channel
                #print("BA Channel column: ",y)
                y_val=np.array([PosToValParis(y)[0],PosToValParis(y)[1]]) #(lat,long) value at index y
                C_next_num=(c[y])*(np.exp(-B*(np.linalg.norm(x_val-y_val)))) #Numerator of the next channel
                C_next_denom=np.dot(c,(exp_mat)[x,:]) #Denominator of the next channel
                C_next[x,y]=(C_next_num)/(C_next_denom) #Next channel
        Channels.append(C_next)
    print("Run time of BA for Paris: %s seconds ---" % (time.time() - start_time))
    return(Channels[R_BA])


In [24]:
#Blahut-Arimoto Algorithm for SF
def BlahutArimotoSF(C0,p0,B,R_BA,Num_X=N_X_sf,Num_Y=N_X_sf):
    ##Inputs:
        #C0=Starting Channel 
        #p0=PDF of source data
        #B=rate-distortion parameter (high values of B implies low avg distortion=QoS and high MI=privacy)
        #R_BA=Number of iterations 
        #Num_X=length of source alphabet
        #Num_Y=length of image alphabet
    ##Output:
        #Returns a channel that computes the minimum MI for a given average distortion

    start_time = time.time()
    #Intermediate storage
    Channels=[] #Storing all the channels till convergence 
    Channels.append(C0) #The first element is the initial matrix

    exp_mat=np.zeros((Num_X,Num_Y)) #Exponential matrix pre-computed 
    for x in range(Num_X):
        x_val=np.array([PosToValSF(x)[0],PosToValSF(x)[1]]) #(lat,long) value at index x
        for y in range(Num_Y):
            y_val=np.array([PosToValSF(y)[0],PosToValSF(y)[1]]) #(lat,long) value at index y
            exp_mat[x,y]=np.exp(-B*(np.linalg.norm(x_val-y_val)))

    for i in range(R_BA): #starting the iteration of BA algorithm
        #print("BA iteration:", i)
        c=np.matmul(p0,Channels[i]) #Computing the output probabilities
        C_next=np.zeros((Num_X,Num_Y)) #Creating a dummy matrix at each step
        for x in range(Num_X): #Rows of the channel
            #print("BA Channel row: ",x)
            x_val=np.array([PosToValSF(x)[0],PosToValSF(x)[1]]) #(lat,long) value at index x
            for y in range(Num_Y): #Columns of the channel
                #print("BA Channel column: ",y)
                y_val=np.array([PosToValSF(y)[0],PosToValSF(y)[1]]) #(lat,long) value at index y
                C_next_num=(c[y])*(np.exp(-B*(np.linalg.norm(x_val-y_val)))) #Numerator of the next channel
                C_next_denom=np.dot(c,(exp_mat)[x,:]) #Denominator of the next channel
                C_next[x,y]=(C_next_num)/(C_next_denom) #Next channel
        Channels.append(C_next)
    print("Run time of BA for SF: %s seconds ---" % (time.time() - start_time))
    return(Channels[R_BA])



In [25]:
#Adding noise to data
#PARIS
def obfuscate_paris(X_samp,C,Y,XLat=X_paris_lat,XLong=X_paris_long,YLat=X_paris_lat,YLong=X_paris_long): #Sanitization of data
    ##Inputs :
        #XLat
        #X_samp = sample of original data
        #C = privacy channel used
        #Y = Codomain of the mechanism/Space of noisy data
    Y_samp_paris=[]#Noisy samples
    for pos_x in range(len(X_samp)):
        x=X_samp[pos_x] #Getting the location data
        lat_x=int(x[0]) #Latitude component of x
        long_x=int(x[1]) #Longitude component of x
        #Finding the position of latitude of the data in the source alphabet for latitudes
        pos_lat_x=int(np.where(X_paris_lat==lat_x)[0][0]) 
        #Finding the position of longitude of the data in the source alphabet for longitudes
        pos_long_x=int(np.where(X_paris_long==long_x)[0][0]) 
        pos_x_1D=ValToPosParis(pos_lat_x,pos_long_x,XLong) #Linear index of the (lat,long) pair #!!!!!!!!! NEEDS TO BE CHAMGED FOR PARIS AND SF DATASETS
        #Adding noise
        prob_x=C[pos_x_1D,:] #x is sanitized wrt the PDF of x'th (in linear index) row of the channel 
        #Note Y is a m*n x 2 array, each row having a (lat,long) pair
        y_1D=int(random.choices(list(range(len(Y))),prob_x,k=1)[0]) #!!!!!!!!! NEEDS TO BE CHAMGED FOR PARIS AND SF DATASETS
        #Extracting latitude, longitude components from y_1D
        y_lat=PosToValParis(y_1D,YLat,YLong)[0]
        y_long=PosToValParis(y_1D,YLat,YLong)[1]
        Y_samp_paris.append(np.array([y_lat,y_long]))
    Y_samp_paris=np.array(Y_samp_paris)
    
    #Computing empirical probabilities of the noisy data
    q_freq_paris=np.array([]) #Empirical probability of noisy_samp
    N_Y_paris=len(Y)
    for pos_y in range(N_Y_paris):
        count_y=0 #Count of element y in noisy locations
        y=Y[pos_y]
        print("Obfuscating: Progress {a} out of {b}".format(a=pos_y, b=N_Y_paris))
        Y_truth=(Y_samp_paris==y)
        for nv in (Y_truth):
            count_y=count_y+(np.all(nv))
        #empirical probability of y in noisy_samp
        q_freq_paris=np.append(q_freq_paris,[count_y/(len(Y_samp_paris))])
    return Y_samp_paris, q_freq_paris #Returning the noisy data (in 2D form) and their empirical probabilities 

#SF
def obfuscate_sf(X_samp,C,Y,XLat=X_sf_lat,XLong=X_sf_long,YLat=X_sf_lat,YLong=X_sf_long): #Sanitization of data
    ##Inputs :
        #X_samp = sample of original data
        #C = privacy channel used
        #Y = Codomain of the mechanism/Space of noisy data
    Y_samp_sf=[] #Noisy samples
    for pos_x in range(len(X_samp)):
        x=X_samp[pos_x] #Getting the location data
        lat_x=int(x[0]) #Latitude component of x
        long_x=int(x[1]) #Longitude component of x
        #Finding the position of latitude of the data in the source alphabet for latitudes
        pos_lat_x=int(np.where(XLat==lat_x)[0][0]) 
        #Finding the position of longitude of the data in the source alphabet for longitudes
        pos_long_x=int(np.where(XLong==long_x)[0][0]) 
        pos_x_1D=ValToPosSF(pos_lat_x,pos_long_x,XLong) #Linear index of the (lat,long) pair #!!!!!!!!! NEEDS TO BE CHAMGED FOR PARIS AND SF DATASETS
        #Adding noise
        prob_x=C[pos_x_1D,:] #x is sanitized wrt the PDF of x'th (in linear index) row of the channel 
        #Note Y is a m*n x 2 array, each row having a (lat,long) pair
        y_1D=int(random.choices(list(range(len(Y))),prob_x,k=1)[0]) #!!!!!!!!! NEEDS TO BE CHAMGED FOR PARIS AND SF DATASETS
        #Extracting latitude, longitude components from y_1D
        y_lat=PosToValSF(y_1D,YLat,YLong)[0]
        y_long=PosToValSF(y_1D,YLat,YLong)[1]
        Y_samp_sf.append(np.array([y_lat,y_long]))
    Y_samp_sf=np.array(Y_samp_sf)
    
    #Computing empirical probabilities of the noisy data
    q_freq_sf=np.array([]) #Empirical probability of noisy_samp
    N_Y_sf=len(Y)
    for pos_y in range(N_Y_sf):
        y=Y[pos_y]
        count_y=0 #Count of element y in noisy locations
        print("Obfuscating: Progress {a} out of {b}".format(a=pos_y, b=N_Y_sf))
        Y_truth=(Y_samp_sf==y)
        for nv in (Y_truth):
            count_y=count_y+(np.all(nv))
        #empirical probability of y in noisy_samp
        q_freq_sf=np.append(q_freq_sf,[count_y/(len(Y_samp_sf))])
    return Y_samp_sf, q_freq_sf #Returning the noisy data (in 2D form) and their empirical probabilities 




In [26]:

#Iterative Bayesian Update
def IBU(p0,q,C,R_IBU,X,Y): #Estimating original distribution
    ##Inputs :
        #p0 = initial guess for the original PDF (full support)
        #q = the empirical PDF of the observed (noisy) data
        #C = the privacy mechanism used
        #R_IBU = number of iterationss' between consequtive iteration
        #X = domain of original data
        #Y = domain of noisy data
    ##Output :
        #Estimate of the original PDF
    start_time = time.time()
    Num_X=len(X)
    Num_Y=len(Y)
    PDFs=[p0] #List of estimated distributions
    for i in tqdm(range(R_IBU)):
        #print ("IBU iteration:", i)
        p=PDFs[i]
        p_next=np.zeros(Num_X)
        for x in range(len(X)): #computing the x'th component of the estimated PDF
            sum_ibu=0
            for y in range(len(Y)):
                num=q[y]*C[x,y]*p[x]
                denom=0
                for z in range(len(X)):
                    denom+=(p[z]*C[z,y])
                sum_ibu=(sum_ibu)+(num/denom)
            p_next[x]=sum_ibu
        PDFs.append(p_next)
    print("Run time of IBU: %s seconds ---" % (time.time() - start_time))
    return PDFs, PDFs[R_IBU]


In [27]:
#CREATING A LINEAR PDF FOR 2D DATA for Paris
p0_paris=np.array([]) #Empirical probability of paris data
N_X_paris=len(X_paris)
for pos_x in tqdm(range(N_X_paris)):
    x=X_paris[pos_x]
    #print("Progress {a} out of {b}".format(a=pos_x, b=N_X_paris))
    freq_x=0 #empirical probability of x in Paris data
    for y in paris_loc:
        comp=(x==y)
        if np.all(comp): #Testing if both elements of x and y are the same
            freq_x+=1
    p0_paris=np.append(p0_paris,[freq_x])
p0_paris=p0_paris/(N_paris) #Normalising frequencies to give empirical probabilities



100%|█████████████████████████████████████████| 165/165 [00:13<00:00, 11.80it/s]


In [28]:
#CREATING A LINEAR PDF FOR 2D DATA for SF
p0_sf=np.array([]) #Empirical probability of paris data
N_X_sf=len(X_sf)
for pos_x in tqdm(range(N_X_sf)):
    x=X_sf[pos_x]
    #print("Progress {a} out of {b}".format(a=pos_x, b=N_X_sf))
    freq_x=0 #empirical probability of x in Paris data
    for y in sf_loc:
        comp=(x==y)
        if np.all(comp): #Testing if both elements of x and y are the same
            freq_x+=1
    p0_sf=np.append(p0_sf,[freq_x])
p0_sf=p0_sf/(N_sf) #Normalising frequencies to give empirical probabilities

100%|█████████████████████████████████████████| 384/384 [06:53<00:00,  1.08s/it]


In [32]:
#Reshaping and initualizing uniform channels
C0_paris=np.reshape(np.array([1/N_X_paris]*(N_X_paris**2)),(N_X_paris,N_X_paris)) #Initial channel to start BA for Paris
p0_paris_unif=np.array([1/N_X_paris]*N_X_paris) #Initial guess of original distribution of Paris
C0_sf=np.reshape(np.array([1/N_X_sf]*(N_X_sf**2)),(N_X_sf,N_X_sf)) #Initial channel to start BA for SF
p0_sf_unif=np.array([1/N_X_sf]*N_X_sf) #Initial guess of original distribution of SF


In [34]:
#############################################################################
##Extracting different channels for locations of 0,high, and isolated probabilities
#Testing for different slope parameters
RDSlope=np.array([0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2,2.2,2.4,2.6,2.8,3,
                  3.2,3.4,3.6,3.8,4,4.2,4.4,4.6,4.8,5])
eps_Oya=2*RDSlope
#Empirical tight epsilons for Paris 
eps_tight_paris=np.array([0.392818028068501,
0.7813925715019457,
1.1605111783813877,
1.522997813452586,
1.9201735805617113,
2.318468681063899,
2.7235710043448096,
3.131913318373869,
3.539017362258331,
3.946218013335979,
4.353319497725823,
4.7597883617851595,
5.1650376600730725,
5.568215644792336,
5.967803359717294,
6.3610873573101845,
6.741703436218012,
7.094334323347863,
7.398355371524807,
7.706459118837066,
8.109656883411978,
8.512103108219597,
8.914469010971294,
9.316911956393886,
9.719415985071155
])
#Empirical tight epsilons for SF
eps_tight_sf=np.array([0.3986617268975043,
0.7883813903959236,
1.1750263394594984,
1.5677798334718995,
1.9816354129563862,
2.3881286826627117,
2.7901762206493075,
3.18914410820109,
3.587243587425301,
3.986583121714903,
4.3873489652913085,
4.78887068465851,
5.1906269545833394,
5.59231245598343,
5.993791505639457,
6.39501755552611,
6.795978710824691,
7.196667510002639,
7.597048656480873,
7.997072913065153,
8.3968159232846,
8.796488394614757,
9.19621125393365,
9.595978444142562,
9.99579069328416
])

RDSlope_short=np.array([0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2,2.2,2.4])
eps_Oya_short=2*RDSlope_short
eps_tight_paris_short=np.array([0.392818028068501,
0.7813925715019457,
1.1605111783813877,
1.522997813452586,
1.9201735805617113,
2.318468681063899,
2.7235710043448096,
3.131913318373869,
3.539017362258331,
3.946218013335979,
4.353319497725823,
4.7597883617851595
])
#Empirical tight epsilons for SF
eps_tight_sf_short=np.array([0.3986617268975043,
0.7883813903959236,
1.1750263394594984,
1.5677798334718995,
1.9816354129563862,
2.3881286826627117,
2.7901762206493075,
3.18914410820109,
3.587243587425301,
3.986583121714903,
4.3873489652913085,
4.78887068465851
])

In [89]:
#####EXPERIMENTS WITH ARTIFICIAL ISLAND IN PARIS!!!!
p0_paris_island=np.array([0.00069458225838,0.02689025600318,0.02133359793610,0.01111331613415,0.00218297281207,0.00992260369121,0.01488390553681,0.02153205000992,0.02302044056360,0.00654891843620,0.00535820599325,0.00208374677515,0.00059535622147,0.00089303433221,0.00039690414765,
0.00079380829530,0.00277832903354,0.02034133756698,0.01944830323477,0.01121254217107,0.01160944631871,0.024458028,0.029121652,0.029320104,0.03517444,0.02465648,0.00674737051002,0.00178606866442,0.00049613018456,0.00049613018456,
0.00506052788252,0.00674737051002,0.00476284977178,0.01109148640603,0.00575511014090,0.01081563802342,0.01448700138916,0.01468545346299,0.01299861083548,0.01885294701330,0.00377058940266,0.00535820599325,0.00416749355031,0.00297678110736,0.00416749355031,
0.00704504862076,0.00119071244295,0.00039690414765,0.00138916451677,0.00337368525501,0.00248065092280,0.00486207580869,0.05288747767414,0.01706687834888,0.02609644770788,0.01280015876166,0.00446517166104,0.00585433617781,0.00079380829530,0.00059535622147,
0.00168684262751,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00714427465767,0.01369319309387,0.01131176820798,0.01617384401667,0.01121254217107,0.00853343917444,0.00694582258385,0.00158761659059,0.00257987695971,
0.00188529470133,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00426671958722,0.03651518158365,0.02331811867434,0.02252431037904,0.00734272673149,0.00188529470133,0.00932724746974,0.01220480254019,0.00228219884898,
0.00039690414765,0.00000000000000,0.00000000000000,0.01,0.00000000000000,0.00000000000000,0.00238142488589,0.00714427465767,0.00754117880532,0.00625124032546,0.00496130184560,0.00019845207382,0.00049613018456,0.00406826751340,0.00069458225838,
0.00109148640603,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00347291129192,0.00000000000000,0.00019845207382,0.00347291129192,0.00337368525501,0.00188529470133,0.00257987695971,0.00932724746974,0.00099226036912,
0.00565588410399,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00178606866442,0.00029767811074,0.00059535622147,0.00218297281207,0.00039690414765,0.00000000000000,0.00000000000000,0.00069458225838,0.00496130184560,
0.00138916451677,0.00059535622147,0.00039690414765,0.00009922603691,0.00079380829530,0.00049613018456,0.00128993847986,0.00248065092280,0.00000000000000,0.00019845207382,0.00178606866442,0.00089303433221,0.00079380829530,0.00386981543957,0.00168684262751,
0.00089303433221,0.00000000000000,0.00059535622147,0.00009922603691,0.00000000000000,0.00039690414765,0.00019845207382,0.00019845207382,0.00128993847986,0.00138916451677,0.00515975391943,0.00000000000000,0.00049613018456,0.01905139908712,0.03105774955348])


#####EXPERIMENTS WITH ARTIFICIAL ISLAND WITH ADVERSARIAL USERS IN PARIS!!!!
p0_paris_island_adversarial=[np.array([0.00069458225838,0.02689025600318,0.02133359793610,0.01111331613415,0.00218297281207,0.00992260369121,0.01488390553681,0.02153205000992,0.02302044056360,0.00654891843620,0.00535820599325,0.00208374677515,0.00059535622147,0.00089303433221,0.00039690414765,
0.00079380829530,0.00277832903354,0.02034133756698,0.01944830323477,0.01121254217107,0.01160944631871,0.024458028,0.029121652,0.029320104,0.03517444,0.02465648,0.00674737051002,0.00178606866442,0.00049613018456,0.00049613018456,
0.00506052788252,0.00674737051002,0.00476284977178,0.01109148640603,0.00575511014090,0.01081563802342,0.01448700138916,0.01468545346299,0.01299861083548,0.01885294701330,0.00377058940266,0.00535820599325,0.00416749355031,0.00297678110736,0.00416749355031,
0.00704504862076,0.00119071244295,0.00039690414765,0.00138916451677,0.00337368525501,0.00248065092280,0.00486207580869,0.05288747767414,0.01706687834888,0.02609644770788,0.01280015876166,0.00446517166104,0.00585433617781,0.00079380829530,0.00059535622147,
0.00168684262751,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00714427465767,0.01369319309387,0.01131176820798,0.01617384401667,0.01121254217107,0.00853343917444,0.00694582258385,0.00158761659059,0.00257987695971,
0.00188529470133,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00426671958722,0.03651518158365,0.02331811867434,0.02252431037904,0.00734272673149,0.00188529470133,0.00932724746974,0.01220480254019,0.00228219884898,
0.00039690414765,0.00000000000000,0.00000000000000,0.01,0.00000000000000,0.00000000000000,0.00238142488589,0.00714427465767,0.00754117880532,0.00625124032546,0.00496130184560,0.00019845207382,0.00049613018456,0.00406826751340,0.00069458225838,
0.00109148640603,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00347291129192,0.00000000000000,0.00019845207382,0.00347291129192,0.00337368525501,0.00188529470133,0.00257987695971,0.00932724746974,0.00099226036912,
0.00565588410399,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00178606866442,0.00029767811074,0.00059535622147,0.00218297281207,0.00039690414765,0.00000000000000,0.00000000000000,0.00069458225838,0.00496130184560,
0.00138916451677,0.00059535622147,0.00039690414765,0.00009922603691,0.00079380829530,0.00049613018456,0.00128993847986,0.00248065092280,0.00000000000000,0.00019845207382,0.00178606866442,0.00089303433221,0.00079380829530,0.00386981543957,0.00168684262751,
0.00089303433221,0.00000000000000,0.00059535622147,0.00009922603691,0.00000000000000,0.00039690414765,0.00019845207382,0.00019845207382,0.00128993847986,0.00138916451677,0.00515975391943,0.00000000000000,0.00049613018456,0.01905139908712,0.03105774955348]),
                            
np.array([0.00069458225838,0.02689025600318,0.02133359793610,0.01111331613415,0.00218297281207,0.00992260369121,0.01488390553681,0.02153205000992,0.02302044056360,0.00654891843620,0.00535820599325,0.00208374677515,0.00059535622147,0.00089303433221,0.00039690414765,
0.00079380829530,0.00277832903354,0.02034133756698,0.01944830323477,0.01121254217107,0.01160944631871,0.024458028,0.029121652,0.029320104,0.03517444,0.02465648,0.00674737051002,0.00178606866442,0.00049613018456,0.00049613018456,
0.00506052788252,0.00674737051002,0.00476284977178,0.01109148640603,0.00575511014090,0.01081563802342,0.01448700138916,0.01468545346299,0.01299861083548,0.01885294701330,0.00377058940266,0.00535820599325,0.00416749355031,0.00297678110736,0.00416749355031,
0.00704504862076,0.00119071244295,0.00039690414765,0.00138916451677,0.00337368525501,0.00248065092280,0.00486207580869,0.05288747767414,0.01706687834888,0.02609644770788,0.01280015876166,0.00446517166104,0.00585433617781,0.00079380829530,0.00059535622147,
0.00168684262751,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00714427465767,0.01369319309387,0.01131176820798,0.01617384401667,0.01121254217107,0.00853343917444,0.00694582258385,0.00158761659059,0.00257987695971,
0.00188529470133,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00426671958722,0.03651518158365,0.02331811867434,0.02252431037904,0.00734272673149,0.00188529470133,0.00932724746974,0.01220480254019,0.00228219884898,
0.00039690414765,0.00000000000000,0.00000000000000,0.015,0.00000000000000,0.00000000000000,0.00238142488589,0.00714427465767,0.00754117880532,0.00625124032546,0.00496130184560,0.00019845207382,0.00049613018456,0.00406826751340,0.00069458225838,
0.00109148640603,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00347291129192,0.00000000000000,0.00019845207382,0.00347291129192,0.00337368525501,0.00188529470133,0.00257987695971,0.00932724746974,0.00099226036912,
0.00565588410399,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00178606866442,0.00029767811074,0.00059535622147,0.00218297281207,0.00039690414765,0.00000000000000,0.00000000000000,0.00069458225838,0.00496130184560,
0.00138916451677,0.00059535622147,0.00039690414765,0.00009922603691,0.00079380829530,0.00049613018456,0.00128993847986,0.00248065092280,0.00000000000000,0.00019845207382,0.00178606866442,0.00089303433221,0.00079380829530,0.00386981543957,0.00168684262751,
0.00089303433221,0.00000000000000,0.00059535622147,0.00009922603691,0.00000000000000,0.00039690414765,0.00019845207382,0.00019845207382,0.00128993847986,0.00138916451677,0.00515975391943,0.00000000000000,0.00049613018456,0.01905139908712,0.03105774955348]),
                            
np.array([0.00069458225838,0.02689025600318,0.02133359793610,0.01111331613415,0.00218297281207,0.00992260369121,0.01488390553681,0.02153205000992,0.02302044056360,0.00654891843620,0.00535820599325,0.00208374677515,0.00059535622147,0.00089303433221,0.00039690414765,
0.00079380829530,0.00277832903354,0.02034133756698,0.01944830323477,0.01121254217107,0.01160944631871,0.024458028,0.029121652,0.029320104,0.03517444,0.02465648,0.00674737051002,0.00178606866442,0.00049613018456,0.00049613018456,
0.00506052788252,0.00674737051002,0.00476284977178,0.01109148640603,0.00575511014090,0.01081563802342,0.01448700138916,0.01468545346299,0.01299861083548,0.01885294701330,0.00377058940266,0.00535820599325,0.00416749355031,0.00297678110736,0.00416749355031,
0.00704504862076,0.00119071244295,0.00039690414765,0.00138916451677,0.00337368525501,0.00248065092280,0.00486207580869,0.05288747767414,0.01706687834888,0.02609644770788,0.01280015876166,0.00446517166104,0.00585433617781,0.00079380829530,0.00059535622147,
0.00168684262751,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00714427465767,0.01369319309387,0.01131176820798,0.01617384401667,0.01121254217107,0.00853343917444,0.00694582258385,0.00158761659059,0.00257987695971,
0.00188529470133,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00426671958722,0.03651518158365,0.02331811867434,0.02252431037904,0.00734272673149,0.00188529470133,0.00932724746974,0.01220480254019,0.00228219884898,
0.00039690414765,0.00000000000000,0.00000000000000,0.02,0.00000000000000,0.00000000000000,0.00238142488589,0.00714427465767,0.00754117880532,0.00625124032546,0.00496130184560,0.00019845207382,0.00049613018456,0.00406826751340,0.00069458225838,
0.00109148640603,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00347291129192,0.00000000000000,0.00019845207382,0.00347291129192,0.00337368525501,0.00188529470133,0.00257987695971,0.00932724746974,0.00099226036912,
0.00565588410399,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00178606866442,0.00029767811074,0.00059535622147,0.00218297281207,0.00039690414765,0.00000000000000,0.00000000000000,0.00069458225838,0.00496130184560,
0.00138916451677,0.00059535622147,0.00039690414765,0.00009922603691,0.00079380829530,0.00049613018456,0.00128993847986,0.00248065092280,0.00000000000000,0.00019845207382,0.00178606866442,0.00089303433221,0.00079380829530,0.00386981543957,0.00168684262751,
0.00089303433221,0.00000000000000,0.00059535622147,0.00009922603691,0.00000000000000,0.00039690414765,0.00019845207382,0.00019845207382,0.00128993847986,0.00138916451677,0.00515975391943,0.00000000000000,0.00049613018456,0.01905139908712,0.03105774955348]),
                            
                        
np.array([0.00069458225838,0.02689025600318,0.02133359793610,0.01111331613415,0.00218297281207,0.00992260369121,0.01488390553681,0.02153205000992,0.02302044056360,0.00654891843620,0.00535820599325,0.00208374677515,0.00059535622147,0.00089303433221,0.00039690414765,
0.00079380829530,0.00277832903354,0.02034133756698,0.01944830323477,0.01121254217107,0.01160944631871,0.024458028,0.029121652,0.029320104,0.03517444,0.02465648,0.00674737051002,0.00178606866442,0.00049613018456,0.00049613018456,
0.00506052788252,0.00674737051002,0.00476284977178,0.01109148640603,0.00575511014090,0.01081563802342,0.01448700138916,0.01468545346299,0.01299861083548,0.01885294701330,0.00377058940266,0.00535820599325,0.00416749355031,0.00297678110736,0.00416749355031,
0.00704504862076,0.00119071244295,0.00039690414765,0.00138916451677,0.00337368525501,0.00248065092280,0.00486207580869,0.05288747767414,0.01706687834888,0.02609644770788,0.01280015876166,0.00446517166104,0.00585433617781,0.00079380829530,0.00059535622147,
0.00168684262751,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00714427465767,0.01369319309387,0.01131176820798,0.01617384401667,0.01121254217107,0.00853343917444,0.00694582258385,0.00158761659059,0.00257987695971,
0.00188529470133,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00426671958722,0.03651518158365,0.02331811867434,0.02252431037904,0.00734272673149,0.00188529470133,0.00932724746974,0.01220480254019,0.00228219884898,
0.00039690414765,0.00000000000000,0.00000000000000,0.025,0.00000000000000,0.00000000000000,0.00238142488589,0.00714427465767,0.00754117880532,0.00625124032546,0.00496130184560,0.00019845207382,0.00049613018456,0.00406826751340,0.00069458225838,
0.00109148640603,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00347291129192,0.00000000000000,0.00019845207382,0.00347291129192,0.00337368525501,0.00188529470133,0.00257987695971,0.00932724746974,0.00099226036912,
0.00565588410399,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00178606866442,0.00029767811074,0.00059535622147,0.00218297281207,0.00039690414765,0.00000000000000,0.00000000000000,0.00069458225838,0.00496130184560,
0.00138916451677,0.00059535622147,0.00039690414765,0.00009922603691,0.00079380829530,0.00049613018456,0.00128993847986,0.00248065092280,0.00000000000000,0.00019845207382,0.00178606866442,0.00089303433221,0.00079380829530,0.00386981543957,0.00168684262751,
0.00089303433221,0.00000000000000,0.00059535622147,0.00009922603691,0.00000000000000,0.00039690414765,0.00019845207382,0.00019845207382,0.00128993847986,0.00138916451677,0.00515975391943,0.00000000000000,0.00049613018456,0.01905139908712,0.03105774955348]),
                            
np.array([0.00069458225838,0.02689025600318,0.02133359793610,0.01111331613415,0.00218297281207,0.00992260369121,0.01488390553681,0.02153205000992,0.02302044056360,0.00654891843620,0.00535820599325,0.00208374677515,0.00059535622147,0.00089303433221,0.00039690414765,
0.00079380829530,0.00277832903354,0.02034133756698,0.01944830323477,0.01121254217107,0.01160944631871,0.024458028,0.029121652,0.029320104,0.03517444,0.02465648,0.00674737051002,0.00178606866442,0.00049613018456,0.00049613018456,
0.00506052788252,0.00674737051002,0.00476284977178,0.01109148640603,0.00575511014090,0.01081563802342,0.01448700138916,0.01468545346299,0.01299861083548,0.01885294701330,0.00377058940266,0.00535820599325,0.00416749355031,0.00297678110736,0.00416749355031,
0.00704504862076,0.00119071244295,0.00039690414765,0.00138916451677,0.00337368525501,0.00248065092280,0.00486207580869,0.05288747767414,0.01706687834888,0.02609644770788,0.01280015876166,0.00446517166104,0.00585433617781,0.00079380829530,0.00059535622147,
0.00168684262751,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00714427465767,0.01369319309387,0.01131176820798,0.01617384401667,0.01121254217107,0.00853343917444,0.00694582258385,0.00158761659059,0.00257987695971,
0.00188529470133,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00426671958722,0.03651518158365,0.02331811867434,0.02252431037904,0.00734272673149,0.00188529470133,0.00932724746974,0.01220480254019,0.00228219884898,
0.00039690414765,0.00000000000000,0.00000000000000,0.03,0.00000000000000,0.00000000000000,0.00238142488589,0.00714427465767,0.00754117880532,0.00625124032546,0.00496130184560,0.00019845207382,0.00049613018456,0.00406826751340,0.00069458225838,
0.00109148640603,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00347291129192,0.00000000000000,0.00019845207382,0.00347291129192,0.00337368525501,0.00188529470133,0.00257987695971,0.00932724746974,0.00099226036912,
0.00565588410399,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00178606866442,0.00029767811074,0.00059535622147,0.00218297281207,0.00039690414765,0.00000000000000,0.00000000000000,0.00069458225838,0.00496130184560,
0.00138916451677,0.00059535622147,0.00039690414765,0.00009922603691,0.00079380829530,0.00049613018456,0.00128993847986,0.00248065092280,0.00000000000000,0.00019845207382,0.00178606866442,0.00089303433221,0.00079380829530,0.00386981543957,0.00168684262751,
0.00089303433221,0.00000000000000,0.00059535622147,0.00009922603691,0.00000000000000,0.00039690414765,0.00019845207382,0.00019845207382,0.00128993847986,0.00138916451677,0.00515975391943,0.00000000000000,0.00049613018456,0.01905139908712,0.03105774955348]),
                            
np.array([0.00069458225838,0.02689025600318,0.02133359793610,0.01111331613415,0.00218297281207,0.00992260369121,0.01488390553681,0.02153205000992,0.02302044056360,0.00654891843620,0.00535820599325,0.00208374677515,0.00059535622147,0.00089303433221,0.00039690414765,
0.00079380829530,0.00277832903354,0.02034133756698,0.01944830323477,0.01121254217107,0.01160944631871,0.024458028,0.029121652,0.029320104,0.03517444,0.02465648,0.00674737051002,0.00178606866442,0.00049613018456,0.00049613018456,
0.00506052788252,0.00674737051002,0.00476284977178,0.01109148640603,0.00575511014090,0.01081563802342,0.01448700138916,0.01468545346299,0.01299861083548,0.01885294701330,0.00377058940266,0.00535820599325,0.00416749355031,0.00297678110736,0.00416749355031,
0.00704504862076,0.00119071244295,0.00039690414765,0.00138916451677,0.00337368525501,0.00248065092280,0.00486207580869,0.05288747767414,0.01706687834888,0.02609644770788,0.01280015876166,0.00446517166104,0.00585433617781,0.00079380829530,0.00059535622147,
0.00168684262751,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00714427465767,0.01369319309387,0.01131176820798,0.01617384401667,0.01121254217107,0.00853343917444,0.00694582258385,0.00158761659059,0.00257987695971,
0.00188529470133,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00426671958722,0.03651518158365,0.02331811867434,0.02252431037904,0.00734272673149,0.00188529470133,0.00932724746974,0.01220480254019,0.00228219884898,
0.00039690414765,0.00000000000000,0.00000000000000,0.035,0.00000000000000,0.00000000000000,0.00238142488589,0.00714427465767,0.00754117880532,0.00625124032546,0.00496130184560,0.00019845207382,0.00049613018456,0.00406826751340,0.00069458225838,
0.00109148640603,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00347291129192,0.00000000000000,0.00019845207382,0.00347291129192,0.00337368525501,0.00188529470133,0.00257987695971,0.00932724746974,0.00099226036912,
0.00565588410399,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00178606866442,0.00029767811074,0.00059535622147,0.00218297281207,0.00039690414765,0.00000000000000,0.00000000000000,0.00069458225838,0.00496130184560,
0.00138916451677,0.00059535622147,0.00039690414765,0.00009922603691,0.00079380829530,0.00049613018456,0.00128993847986,0.00248065092280,0.00000000000000,0.00019845207382,0.00178606866442,0.00089303433221,0.00079380829530,0.00386981543957,0.00168684262751,
0.00089303433221,0.00000000000000,0.00059535622147,0.00009922603691,0.00000000000000,0.00039690414765,0.00019845207382,0.00019845207382,0.00128993847986,0.00138916451677,0.00515975391943,0.00000000000000,0.00049613018456,0.01905139908712,0.03105774955348]),
                            
np.array([0.00069458225838,0.02689025600318,0.02133359793610,0.01111331613415,0.00218297281207,0.00992260369121,0.01488390553681,0.02153205000992,0.02302044056360,0.00654891843620,0.00535820599325,0.00208374677515,0.00059535622147,0.00089303433221,0.00039690414765,
0.00079380829530,0.00277832903354,0.02034133756698,0.01944830323477,0.01121254217107,0.01160944631871,0.024458028,0.029121652,0.029320104,0.03517444,0.02465648,0.00674737051002,0.00178606866442,0.00049613018456,0.00049613018456,
0.00506052788252,0.00674737051002,0.00476284977178,0.01109148640603,0.00575511014090,0.01081563802342,0.01448700138916,0.01468545346299,0.01299861083548,0.01885294701330,0.00377058940266,0.00535820599325,0.00416749355031,0.00297678110736,0.00416749355031,
0.00704504862076,0.00119071244295,0.00039690414765,0.00138916451677,0.00337368525501,0.00248065092280,0.00486207580869,0.05288747767414,0.01706687834888,0.02609644770788,0.01280015876166,0.00446517166104,0.00585433617781,0.00079380829530,0.00059535622147,
0.00168684262751,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00714427465767,0.01369319309387,0.01131176820798,0.01617384401667,0.01121254217107,0.00853343917444,0.00694582258385,0.00158761659059,0.00257987695971,
0.00188529470133,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00426671958722,0.03651518158365,0.02331811867434,0.02252431037904,0.00734272673149,0.00188529470133,0.00932724746974,0.01220480254019,0.00228219884898,
0.00039690414765,0.00000000000000,0.00000000000000,0.04,0.00000000000000,0.00000000000000,0.00238142488589,0.00714427465767,0.00754117880532,0.00625124032546,0.00496130184560,0.00019845207382,0.00049613018456,0.00406826751340,0.00069458225838,
0.00109148640603,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00347291129192,0.00000000000000,0.00019845207382,0.00347291129192,0.00337368525501,0.00188529470133,0.00257987695971,0.00932724746974,0.00099226036912,
0.00565588410399,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00178606866442,0.00029767811074,0.00059535622147,0.00218297281207,0.00039690414765,0.00000000000000,0.00000000000000,0.00069458225838,0.00496130184560,
0.00138916451677,0.00059535622147,0.00039690414765,0.00009922603691,0.00079380829530,0.00049613018456,0.00128993847986,0.00248065092280,0.00000000000000,0.00019845207382,0.00178606866442,0.00089303433221,0.00079380829530,0.00386981543957,0.00168684262751,
0.00089303433221,0.00000000000000,0.00059535622147,0.00009922603691,0.00000000000000,0.00039690414765,0.00019845207382,0.00019845207382,0.00128993847986,0.00138916451677,0.00515975391943,0.00000000000000,0.00049613018456,0.01905139908712,0.03105774955348]),
                            
np.array([0.00069458225838,0.02689025600318,0.02133359793610,0.01111331613415,0.00218297281207,0.00992260369121,0.01488390553681,0.02153205000992,0.02302044056360,0.00654891843620,0.00535820599325,0.00208374677515,0.00059535622147,0.00089303433221,0.00039690414765,
0.00079380829530,0.00277832903354,0.02034133756698,0.01944830323477,0.01121254217107,0.01160944631871,0.024458028,0.029121652,0.029320104,0.03517444,0.02465648,0.00674737051002,0.00178606866442,0.00049613018456,0.00049613018456,
0.00506052788252,0.00674737051002,0.00476284977178,0.01109148640603,0.00575511014090,0.01081563802342,0.01448700138916,0.01468545346299,0.01299861083548,0.01885294701330,0.00377058940266,0.00535820599325,0.00416749355031,0.00297678110736,0.00416749355031,
0.00704504862076,0.00119071244295,0.00039690414765,0.00138916451677,0.00337368525501,0.00248065092280,0.00486207580869,0.05288747767414,0.01706687834888,0.02609644770788,0.01280015876166,0.00446517166104,0.00585433617781,0.00079380829530,0.00059535622147,
0.00168684262751,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00714427465767,0.01369319309387,0.01131176820798,0.01617384401667,0.01121254217107,0.00853343917444,0.00694582258385,0.00158761659059,0.00257987695971,
0.00188529470133,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00426671958722,0.03651518158365,0.02331811867434,0.02252431037904,0.00734272673149,0.00188529470133,0.00932724746974,0.01220480254019,0.00228219884898,
0.00039690414765,0.00000000000000,0.00000000000000,0.045,0.00000000000000,0.00000000000000,0.00238142488589,0.00714427465767,0.00754117880532,0.00625124032546,0.00496130184560,0.00019845207382,0.00049613018456,0.00406826751340,0.00069458225838,
0.00109148640603,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00347291129192,0.00000000000000,0.00019845207382,0.00347291129192,0.00337368525501,0.00188529470133,0.00257987695971,0.00932724746974,0.00099226036912,
0.00565588410399,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00178606866442,0.00029767811074,0.00059535622147,0.00218297281207,0.00039690414765,0.00000000000000,0.00000000000000,0.00069458225838,0.00496130184560,
0.00138916451677,0.00059535622147,0.00039690414765,0.00009922603691,0.00079380829530,0.00049613018456,0.00128993847986,0.00248065092280,0.00000000000000,0.00019845207382,0.00178606866442,0.00089303433221,0.00079380829530,0.00386981543957,0.00168684262751,
0.00089303433221,0.00000000000000,0.00059535622147,0.00009922603691,0.00000000000000,0.00039690414765,0.00019845207382,0.00019845207382,0.00128993847986,0.00138916451677,0.00515975391943,0.00000000000000,0.00049613018456,0.01905139908712,0.03105774955348]),
                            
np.array([0.00069458225838,0.02689025600318,0.02133359793610,0.01111331613415,0.00218297281207,0.00992260369121,0.01488390553681,0.02153205000992,0.02302044056360,0.00654891843620,0.00535820599325,0.00208374677515,0.00059535622147,0.00089303433221,0.00039690414765,
0.00079380829530,0.00277832903354,0.02034133756698,0.01944830323477,0.01121254217107,0.01160944631871,0.024458028,0.029121652,0.029320104,0.03517444,0.02465648,0.00674737051002,0.00178606866442,0.00049613018456,0.00049613018456,
0.00506052788252,0.00674737051002,0.00476284977178,0.01109148640603,0.00575511014090,0.01081563802342,0.01448700138916,0.01468545346299,0.01299861083548,0.01885294701330,0.00377058940266,0.00535820599325,0.00416749355031,0.00297678110736,0.00416749355031,
0.00704504862076,0.00119071244295,0.00039690414765,0.00138916451677,0.00337368525501,0.00248065092280,0.00486207580869,0.05288747767414,0.01706687834888,0.02609644770788,0.01280015876166,0.00446517166104,0.00585433617781,0.00079380829530,0.00059535622147,
0.00168684262751,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00714427465767,0.01369319309387,0.01131176820798,0.01617384401667,0.01121254217107,0.00853343917444,0.00694582258385,0.00158761659059,0.00257987695971,
0.00188529470133,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00426671958722,0.03651518158365,0.02331811867434,0.02252431037904,0.00734272673149,0.00188529470133,0.00932724746974,0.01220480254019,0.00228219884898,
0.00039690414765,0.00000000000000,0.00000000000000,0.05,0.00000000000000,0.00000000000000,0.00238142488589,0.00714427465767,0.00754117880532,0.00625124032546,0.00496130184560,0.00019845207382,0.00049613018456,0.00406826751340,0.00069458225838,
0.00109148640603,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00347291129192,0.00000000000000,0.00019845207382,0.00347291129192,0.00337368525501,0.00188529470133,0.00257987695971,0.00932724746974,0.00099226036912,
0.00565588410399,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00178606866442,0.00029767811074,0.00059535622147,0.00218297281207,0.00039690414765,0.00000000000000,0.00000000000000,0.00069458225838,0.00496130184560,
0.00138916451677,0.00059535622147,0.00039690414765,0.00009922603691,0.00079380829530,0.00049613018456,0.00128993847986,0.00248065092280,0.00000000000000,0.00019845207382,0.00178606866442,0.00089303433221,0.00079380829530,0.00386981543957,0.00168684262751,
0.00089303433221,0.00000000000000,0.00059535622147,0.00009922603691,0.00000000000000,0.00039690414765,0.00019845207382,0.00019845207382,0.00128993847986,0.00138916451677,0.00515975391943,0.00000000000000,0.00049613018456,0.01905139908712,0.03105774955348])]

#########!!!!Finding behaviour of channels for artificial island paris
##Only interested in setting 4 and setting 1

#Setting 4 (vulnerable location -- isolated on an island) index: 93
#Setting 1 (strong location -- crowded around it) index: 52

str_loc_index=52 #Strong location in a crowd
vul_loc_index=93 #Vulnarable location in isolation


In [91]:
#List of the privacy channels for different levels of geo-indistinguishability with adversarial users
##Channels via BA algorithm
Channels_BA_Paris_island_adv={} #BA mechanism for Paris island locations with different levels of adversarial inputs


for i in tqdm(range(len(RDSlope_short))):
    beta=RDSlope_short[i] #Slope parameter for Blahut-Arimoto
    eps_paris=eps_tight_paris_short[i] #Geo-Ind parameter for Paris
    for p in range(len(p0_paris_island_adversarial)):
        C_BA_paris_island=BlahutArimotoParis(C0_paris, p0_paris_island_adversarial[p], B=beta, R_BA=10, Num_X=N_X_paris, Num_Y=N_X_paris) #BA channel for Paris_island
        key="Beta {} Adv {}".format(beta,p)
        Channels_BA_Paris_island_adv[key]=C_BA_paris_island


  0%|                                                    | 0/12 [00:00<?, ?it/s]

Run time of BA for Paris: 2.592609167098999 seconds ---
Run time of BA for Paris: 2.5366017818450928 seconds ---
Run time of BA for Paris: 2.4871039390563965 seconds ---
Run time of BA for Paris: 2.6550698280334473 seconds ---
Run time of BA for Paris: 2.5620779991149902 seconds ---
Run time of BA for Paris: 2.587894916534424 seconds ---
Run time of BA for Paris: 2.8656089305877686 seconds ---
Run time of BA for Paris: 2.7973010540008545 seconds ---



  8%|███▋                                        | 1/12 [00:24<04:26, 24.18s/it]

Run time of BA for Paris: 3.098121166229248 seconds ---
Run time of BA for Paris: 2.7446768283843994 seconds ---
Run time of BA for Paris: 2.5874788761138916 seconds ---
Run time of BA for Paris: 2.5880401134490967 seconds ---
Run time of BA for Paris: 2.538456678390503 seconds ---
Run time of BA for Paris: 2.5356948375701904 seconds ---
Run time of BA for Paris: 2.640584945678711 seconds ---
Run time of BA for Paris: 2.644521951675415 seconds ---
Run time of BA for Paris: 2.5768418312072754 seconds ---



 17%|███████▎                                    | 2/12 [00:47<03:57, 23.74s/it]

...

Run time of BA for Paris: 2.669055938720703 seconds ---
Run time of BA for Paris: 2.611297130584717 seconds ---
Run time of BA for Paris: 2.587986946105957 seconds ---
Run time of BA for Paris: 2.5615649223327637 seconds ---
Run time of BA for Paris: 2.5690219402313232 seconds ---
Run time of BA for Paris: 2.5440709590911865 seconds ---
Run time of BA for Paris: 2.7627270221710205 seconds ---
Run time of BA for Paris: 3.0140888690948486 seconds ---
Run time of BA for Paris: 3.0160398483276367 seconds ---



 92%|███████████████████████████████████████▍   | 11/12 [04:57<00:27, 27.84s/it]

Run time of BA for Paris: 2.6384787559509277 seconds ---
Run time of BA for Paris: 2.5456790924072266 seconds ---
Run time of BA for Paris: 2.673563241958618 seconds ---
Run time of BA for Paris: 2.7809200286865234 seconds ---
Run time of BA for Paris: 2.850675344467163 seconds ---
Run time of BA for Paris: 3.2826550006866455 seconds ---
Run time of BA for Paris: 3.4523539543151855 seconds ---
Run time of BA for Paris: 2.9540610313415527 seconds ---
Run time of BA for Paris: 2.5282201766967773 seconds ---


100%|███████████████████████████████████████████| 12/12 [05:23<00:00, 26.93s/it]

Run time of BA for Paris: 2.562056303024292 seconds ---


In [46]:
Channels_BA_Paris_island_adv


{'Beta 0.2 Adv 0': array([[7.96186340e-04, 2.83263446e-03, 5.87235366e-03, ...,
         3.78585542e-05, 3.53886469e-05, 2.05715693e-05],
        [5.45366000e-04, 2.89455405e-03, 6.00071958e-03, ...,
         3.68319971e-05, 3.46105641e-05, 2.02108599e-05],
        [3.76860036e-04, 2.00020123e-03, 6.18604609e-03, ...,
         3.59299177e-05, 3.39694575e-05, 1.99410582e-05],
        ...,
        [3.81247438e-05, 1.92650726e-04, 5.63808019e-04, ...,
         9.37378321e-04, 8.38626206e-04, 4.68703432e-04],
        [3.62176433e-05, 1.83978631e-04, 5.41722605e-04, ...,
         8.52278927e-04, 1.13750435e-03, 6.35744734e-04],
        [3.48142478e-05, 1.77654765e-04, 5.25859729e-04, ...,
         7.87670933e-04, 1.05127451e-03, 8.76523639e-04]]),
 'Beta 0.2 Adv 1': array([[7.96963227e-04, 2.82473899e-03, 5.84782836e-03, ...,
         3.74369463e-05, 3.48511238e-05, 2.02091296e-05],
        [5.46077316e-04, 2.88743336e-03, 5.97761946e-03, ...,
         3.64337753e-05, 3.40960464e-05, 1.9861

In [94]:

#!!!#!!!#
##Getting the obfuscation probabilities

#Indices of X_paris_wide corresponding to X_paris
L_paris=[]
for i in range(len(X_paris_wide)):
    x1=np.array(X_paris_wide[i])
    for j in range(len((X_paris))):
        x2=np.array(X_paris[j])
        if ((x1==x2).all()):
            L_paris.append(i)


##Setting 1:
##Location of high probability in crowd
#!!PARIS ISLAND
density1_BA_Paris_island_adv={} #List of obfucation PDFs of strong location in Paris island under BA of different privacy level

str_loc_index=52 #Index of strong location in a crowd
str_loc=(PosToValParis(str_loc_index)) #(Latitude, Longitude) of strong location in a crowd


##Setting 4:
##Location of high probability in isolation
#!!PARIS ISLAND
density4_BA_Paris_island_adv={} #List of obfucation PDFs of vulnerable location in Paris island under BA of different privacy level

vul_loc_index=93 #Index of vulnerable location in isolation
vul_loc=(PosToValParis(vul_loc_index)) #(Latitude, Longitude) of vulnerable location in isolation




for i in tqdm(range(len(RDSlope_short))):
    beta=RDSlope_short[i] #Slope parameter for Blahut-Arimoto
    eps_paris=eps_tight_paris_short[i] #Geo-Ind parameter for Paris
    for p in range(len(p0_paris_island_adversarial)):
        key="Beta {} Adv {}".format(beta,p)
        #Calling the channels
        CBAPar=Channels_BA_Paris_island_adv[key]
        ##Blahut-Arimoto obfuscation probabilities
        #Obfuscation location for Paris location in setting 1
        density1_BA_Paris_island_adv[key]=CBAPar[str_loc_index]
        #Obfuscation location for Paris location in setting 4
        density4_BA_Paris_island_adv[key]=CBAPar[vul_loc_index]


100%|████████████████████████████████████████| 12/12 [00:00<00:00, 30596.75it/s]


In [103]:
#Saving 2D obfuscation probabilities
##########~~~~~~~!!!


##~~~~~~Blahut-Arimoto


#Setting 4 -- Isolated point in the map
b=3 #index for beta value  
p=0 #Adversarial Input level
key="Beta {} Adv {}".format(RDSlope_short[b],p)
ob4_BA_paris_beta_0_2=np.reshape(density4_BA_Paris_island_adv[key], (N_X_paris_lat,N_X_paris_long))
w = open('ObfusProb vul Paris BA Beta 0_8 Adv 0.csv', 'w', encoding='utf-8')
for i in range(N_X_paris_lat):
    for j in range(1,N_X_paris_long):
        w.write(str(ob4_BA_paris_beta_0_2[i][j])+' ')
    w.write(str(ob4_BA_paris_beta_0_2[i][N_X_paris_long-1])+'\n')
w.close()

b=3 #index for beta value 
p=1 #Adversarial Input level
key="Beta {} Adv {}".format(RDSlope_short[b],p)
ob4_BA_paris_beta_0_2=np.reshape(density4_BA_Paris_island_adv[key], (N_X_paris_lat,N_X_paris_long))
w = open('ObfusProb vul Paris BA Beta 0_8 Adv 1.csv', 'w', encoding='utf-8')
for i in range(N_X_paris_lat):
    for j in range(1,N_X_paris_long):
        w.write(str(ob4_BA_paris_beta_0_2[i][j])+' ')
    w.write(str(ob4_BA_paris_beta_0_2[i][N_X_paris_long-1])+'\n')
w.close()

b=3 #index for beta value 
p=2 #Adversarial Input level
key="Beta {} Adv {}".format(RDSlope_short[b],p)
ob4_BA_paris_beta_0_2=np.reshape(density4_BA_Paris_island_adv[key], (N_X_paris_lat,N_X_paris_long))
w = open('ObfusProb vul Paris BA Beta 0_8 Adv 2.csv', 'w', encoding='utf-8')
for i in range(N_X_paris_lat):
    for j in range(1,N_X_paris_long):
        w.write(str(ob4_BA_paris_beta_0_2[i][j])+' ')
    w.write(str(ob4_BA_paris_beta_0_2[i][N_X_paris_long-1])+'\n')
w.close()

b=3 #index for beta value 
p=3 #Adversarial Input level
key="Beta {} Adv {}".format(RDSlope_short[b],p)
ob4_BA_paris_beta_0_2=np.reshape(density4_BA_Paris_island_adv[key], (N_X_paris_lat,N_X_paris_long))
w = open('ObfusProb vul Paris BA Beta 0_8 Adv 3.csv', 'w', encoding='utf-8')
for i in range(N_X_paris_lat):
    for j in range(1,N_X_paris_long):
        w.write(str(ob4_BA_paris_beta_0_2[i][j])+' ')
    w.write(str(ob4_BA_paris_beta_0_2[i][N_X_paris_long-1])+'\n')
w.close()

b=3 #index for beta value 
p=4 #Adversarial Input level
key="Beta {} Adv {}".format(RDSlope_short[b],p)
ob4_BA_paris_beta_0_2=np.reshape(density4_BA_Paris_island_adv[key], (N_X_paris_lat,N_X_paris_long))
w = open('ObfusProb vul Paris BA Beta 0_8 Adv 4.csv', 'w', encoding='utf-8')
for i in range(N_X_paris_lat):
    for j in range(1,N_X_paris_long):
        w.write(str(ob4_BA_paris_beta_0_2[i][j])+' ')
    w.write(str(ob4_BA_paris_beta_0_2[i][N_X_paris_long-1])+'\n')
w.close()


#Setting 4 -- Isolated point in the map
b=3 #index for beta value 
p=5 #Adversarial Input level
key="Beta {} Adv {}".format(RDSlope_short[b],p)
ob4_BA_paris_beta_0_2=np.reshape(density4_BA_Paris_island_adv[key], (N_X_paris_lat,N_X_paris_long))
w = open('ObfusProb vul Paris BA Beta 0_8 Adv 5.csv', 'w', encoding='utf-8')
for i in range(N_X_paris_lat):
    for j in range(1,N_X_paris_long):
        w.write(str(ob4_BA_paris_beta_0_2[i][j])+' ')
    w.write(str(ob4_BA_paris_beta_0_2[i][N_X_paris_long-1])+'\n')
w.close()

b=3 #index for beta value 
p=6 #Adversarial Input level
key="Beta {} Adv {}".format(RDSlope_short[b],p)
ob4_BA_paris_beta_0_2=np.reshape(density4_BA_Paris_island_adv[key], (N_X_paris_lat,N_X_paris_long))
w = open('ObfusProb vul Paris BA Beta 0_8 Adv 6.csv', 'w', encoding='utf-8')
for i in range(N_X_paris_lat):
    for j in range(1,N_X_paris_long):
        w.write(str(ob4_BA_paris_beta_0_2[i][j])+' ')
    w.write(str(ob4_BA_paris_beta_0_2[i][N_X_paris_long-1])+'\n')
w.close()

b=3 #index for beta value 
p=7 #Adversarial Input level
key="Beta {} Adv {}".format(RDSlope_short[b],p)
ob4_BA_paris_beta_0_2=np.reshape(density4_BA_Paris_island_adv[key], (N_X_paris_lat,N_X_paris_long))
w = open('ObfusProb vul Paris BA Beta 0_8 Adv 7.csv', 'w', encoding='utf-8')
for i in range(N_X_paris_lat):
    for j in range(1,N_X_paris_long):
        w.write(str(ob4_BA_paris_beta_0_2[i][j])+' ')
    w.write(str(ob4_BA_paris_beta_0_2[i][N_X_paris_long-1])+'\n')
w.close()

b=3 #index for beta value 
p=8 #Adversarial Input level
key="Beta {} Adv {}".format(RDSlope_short[b],p)
ob4_BA_paris_beta_0_2=np.reshape(density4_BA_Paris_island_adv[key], (N_X_paris_lat,N_X_paris_long))
w = open('ObfusProb vul Paris BA Beta 0_8 Adv 8.csv', 'w', encoding='utf-8')
for i in range(N_X_paris_lat):
    for j in range(1,N_X_paris_long):
        w.write(str(ob4_BA_paris_beta_0_2[i][j])+' ')
    w.write(str(ob4_BA_paris_beta_0_2[i][N_X_paris_long-1])+'\n')
w.close()
